In [ ]:
!pip install openai==0.27.8

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 1.7 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import openai
import os
openai.api_base = "https://api.ai-gaochao.cn/v1"
openai.api_key = ""

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_all = pd.read_excel('formula_generated_results.xlsx')

In [ ]:
df = df_all.copy()

In [ ]:
def check_answer_with_gpt3_5(question, generated_answer, correct_answer):
    prompt = f"""
    Based on the given question, generated answer, and correct answer, determine if the generated answer is correct. The generated answer is generally in the last sentence.

    Question: {question}
    Generated Answer: {generated_answer}
    Correct Answer: {correct_answer}

    Answer "Correct" if the generated answer matches the correct answer, otherwise answer "Incorrect".
    """

    while True:
        try:
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "You are an expert in evaluating generated answers."},
                    {"role": "user", "content": prompt}
                ]
            )
            break
        except openai.error.RateLimitError:
            print("Rate limit reached. Waiting for 5 seconds before retrying...")
            time.sleep(5)

    response_text = response['choices'][0]['message']['content'].strip()
    return response_text

def check_ans(df):
    results = []

    for index, row in df.iterrows():
        result = check_answer_with_gpt3_5(row["Question"], row["Generated Answer"], row["Correct Answer"])
        print(f"Question {index + 1} result: {result}")
        results.append(result == "Correct")

    accuracy = (sum(results) / len(results)) * 100
    return accuracy

def main():
    accuracy = check_ans(df)
    print(f'Overall Accuracy: {accuracy:.2f}%')

if __name__ == "__main__":
    main()